# Score mouse regions in human sc data

In [ ]:
import warnings
warnings.filterwarnings('ignore','invalid value encountered in true_divide')

import pandas as pd
import numpy as np
import anndata as ad

import scanpy as sc
import tacco as tc

import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.cm import ScalarMappable

from statsmodels.stats.multitest import multipletests

from statannotations.Annotator import Annotator

In [ ]:
import sys
# Make helper functions available: The notebook expects to be executed either in the sub-workflow directory or in the notebooks directory
sys.path.insert(1, '../'), sys.path.insert(1, '../workflow/'); # prefer to look just one directory up
import helper
sys.path.pop(1), sys.path.pop(1);

get_path = helper.get_paths('human_sc')
figures_folder = get_path('plots')

## Load data and convert the genes to the common MGI homology classes

In [ ]:
tc.tl.setup_orthology_converter(f'{get_path("resources")}/MGI/HOM_AllOrganism.rpt');

In [ ]:
mouse_scrna = ad.read(f'{get_path("resources","mouse_sc")}/scRNAseq.h5ad')
mouse_slideseq = ad.read(f'{get_path("resources","mouse_slideseq")}/slideseq.h5ad')
mouse_slideseq_by_compartment = ad.read(f'{get_path("resources","mouse_slideseq")}/slideseq_by_compartment.h5ad')

In [ ]:
# use only data from sufficiently covered beads
mouse_slideseq = mouse_slideseq[mouse_slideseq.X.sum(axis=1)>=100].copy()
mouse_slideseq_by_compartment = mouse_slideseq_by_compartment[mouse_slideseq_by_compartment.obs['index'].isin(mouse_slideseq.obs.index)].copy()

In [ ]:
mouse_scrna = tc.tl.run_orthology_converter(mouse_scrna, 'mouse', use_synonyms=False) # no synonyms here to keep integer counts (also all counts are already used without synonyms)
mouse_slideseq_by_compartment = tc.tl.run_orthology_converter(mouse_slideseq_by_compartment, 'mouse', use_synonyms=False) # no synonyms here to keep integer counts (also all counts are already used without synonyms)
# remove varm as it breaks anndata if subsetted...
for k in [k for k in mouse_scrna.varm]:
    del mouse_scrna.varm[k]

In [ ]:
mouse_adatas = { 'scRNA': mouse_scrna, 'SlideSeq': mouse_slideseq_by_compartment }

In [ ]:
human_data_sources = [ 'Pelka', 'Becker', 'Zheng', 'Khaliq', 'Che', 'Chen', 'Joanito_3p', 'Joanito_5p' ]

In [ ]:
human_adatas = { source: ad.read(f'{get_path("data")}/{source}.h5ad') for source in human_data_sources }

In [ ]:
for source in human_adatas.keys():
    human_adatas[source] = tc.tl.run_orthology_converter(human_adatas[source], 'human', use_synonyms=True) # use synonyms here as it increases the amount of data used (and the integer nature of the data is irrelevant)

In [ ]:
for source, human_adata in human_adatas.items():
    human_adata.obs['species'] = 'human'
    human_adata.obs['source'] = source

In [ ]:
for source, mouse_adata in mouse_adatas.items():
    mouse_adata.obs['species'] = 'mouse'
    mouse_adata.obs['source'] = source

In [ ]:
all_adatas = {**mouse_adatas, **human_adatas}

In [ ]:
# generate a consistent layer of annotation across datasets
for source, adata in all_adatas.items():
    if 'SampleID' not in adata.obs:
        if 'samples' in adata.obs:
            adata.obs['SampleID'] = adata.obs['samples']
        elif 'sample' in adata.obs:
            adata.obs['SampleID'] = adata.obs['sample']
        elif 'PID' in adata.obs:
            adata.obs['SampleID'] = adata.obs['PID']
        else:
            raise ValueError(f'Unknown sample spec in source {source}!')
    if 'State' not in adata.obs:
        if 'GrossPathology' in adata.obs:
            adata.obs['State'] = adata.obs['GrossPathology']
        elif 'TMMR' in adata.obs:
            adata.obs['State'] = adata.obs['TMMR']
        elif 'Condition' in adata.obs:
            adata.obs['State'] = adata.obs['Condition']
        else:
            raise ValueError(f'Unknown state spec in source {source}!')
    if 'Epithelial' not in adata.obs:
        if 'epithelial' in adata.obs:
            adata.obs['Epithelial'] = adata.obs['epithelial']
        elif 'compartment' in adata.obs:
            adata.obs['Epithelial'] = adata.obs['compartment'].isin(['Epi','Epithelial','epi','epithelial'])
        elif 'pelka_epithelial' in adata.obs:
            adata.obs['Epithelial'] = adata.obs['pelka_epithelial']
        else:
            raise ValueError(f'Unknown sample spec in source {source}!')
    adata.obs['source-State'] = adata.obs['source'].astype(str) + ': ' + adata.obs['State'].astype(str)

In [ ]:
# filter all datasets to a common gene set
filtered_adatas = tc.pp.filter([*human_adatas.values(),*mouse_adatas.values()], return_view=False, remove_constant_genes=True)

## Load CRC classification results for full pseudobulk samples

In [ ]:
CMS_classes = pd.read_csv(f'{get_path("data")}/CMS_classification.tsv',sep='\t')

# generate pseudobulk samples for the epithelial compartment

In [ ]:
all_data = [ adata[adata.obs['Epithelial']] for adata in filtered_adatas ]
def prep_pseudobulk(adata):
    pseudobulk = tc.tl.merge_observations(adata, 'SampleID')
    pseudobulk.obs['SampleID'] = pseudobulk.obs['SampleID'].astype(str)
    pseudobulk.obs = pseudobulk.obs.set_index('SampleID')
    return pseudobulk
all_pseudobulk = [ prep_pseudobulk(adata) for adata in all_data ]

In [ ]:
adata_pseudobulk = ad.concat(all_pseudobulk)

In [ ]:
tc.utils.merge_annotation(adata_pseudobulk, 'source', mapping={
    'mouse-10x3p': ['scRNA',],
    'mouse-SlideSeq': ['SlideSeq',],
    'human-10x3p': ['Pelka','Zheng','Che','Joanito_3p'],
    'human-10x5p': ['Khaliq','Joanito_5p'],
    'human-inDrop': ['Chen',],
    'human-snRNA': ['Becker',],
}, result_key='batch')
adata_pseudobulk.obs['reference-normal'] = adata_pseudobulk.obs['source-State'].isin(['scRNA: normal','SlideSeq: normal','Zheng: normal','Joanito_5p: Normal','Chen: NL','Becker: Normal',])
batch_pseudobulks = [adata_pseudobulk[df.index].copy() for k,df in adata_pseudobulk.obs.groupby('batch')]
filtered_reference_normals = tc.pp.filter([batch_pseudobulk[batch_pseudobulk.obs['reference-normal']] for batch_pseudobulk in batch_pseudobulks], min_cells_per_gene=2, remove_constant_genes=True, remove_zero_cells=True, return_view=True)
batch_pseudobulks = [batch_pseudobulk[:,filtered_reference_normals[0].var.index].copy() for batch_pseudobulk in batch_pseudobulks]
target_normal = adata_pseudobulk[adata_pseudobulk.obs['source-State'] == 'Zheng: normal', filtered_reference_normals[0].var.index].copy()
for batch_pseudobulk in batch_pseudobulks:
    factors = tc.pp.normalize_platform(batch_pseudobulk[batch_pseudobulk.obs['reference-normal']], target_normal, inplace=False, return_rescaling_factors=True)
    tc.utils.scale_counts(batch_pseudobulk, factors)
corrected_pseudobulk = ad.concat(batch_pseudobulks)

In [ ]:
def norm_pseudobulk(adata, min_counts=1e5):
    adata = tc.pp.filter(adata, min_counts_per_cell=min_counts, return_view=False, assume_valid_counts=True)
    adata.X = tc.tl.get_contributions(adata, None, value_location='X', normalization='clr', reduction=None, assume_counts=True).to_numpy()
    return adata
normed_pseudobulk = norm_pseudobulk(corrected_pseudobulk)

# Get DE genes for the regions

In [ ]:
# score regions; using epithelial data only
def DEG_regions(selected_regions):
    _sdata = mouse_slideseq_by_compartment[mouse_slideseq_by_compartment.obs['Epithelial'],normed_pseudobulk.var.index]
    
    group_key = 'region'

    enrichments = helper.marker_genes(_sdata, group_key, rungo=False, restrict_groups=selected_regions)

    gene_lists = {region:df.sort_values('p_fisher_fdr_bh')['value'].str.upper() for region, df in enrichments.groupby(group_key) if len(df)>0}
    return gene_lists
gene_lists = {}
gene_lists['aR'] = DEG_regions([ r for r in mouse_slideseq_by_compartment.obs['region'].cat.categories ])
gene_lists['mR'] = DEG_regions([ r for r in mouse_slideseq_by_compartment.obs['region'].cat.categories if 'Malignant' in r ])

# Score pseudobulk data with the DE genes for the regions

In [ ]:
def score_adata(adata, score, nDEG=200):
    for region,genes in gene_lists[score].items():
        adata.obs[f'{region}_{score}score'] = np.asarray(adata[:,genes.head(nDEG)].X.mean(axis=1))

In [ ]:
for k in gene_lists.keys():
    score_adata(normed_pseudobulk, k, nDEG=200)

In [ ]:
def umap_dataframe(adatas, score, keep=['State','species','source','source-State','batch','reference-normal']):
    keys=[f'{region}_{score}score' for region in gene_lists[score].keys()]
    if not isinstance(adatas, list):
        adatas = [adatas]
    score_data = ad.AnnData(pd.concat([ adata.obs[keys] for adata in adatas ]))
    if keep is not None:
        for k in keep:
            for adata in adatas:
                if k in adata.obs.columns:
                    if k not in score_data.obs:
                        score_data.obs[k] = None
                    score_data.obs.loc[adata.obs.index,k] = adata.obs[k]
    
    # correct for in-species prediction bias
    batch_score_datas = [score_data[df.index].copy() for k,df in score_data.obs.groupby('species')]
    for batch_score_data in batch_score_datas:
        mean = batch_score_data.X.mean(axis=0)
        std = batch_score_data.X.std(axis=0)
        batch_score_data.X -= mean
        batch_score_data.X /= std
    score_data = ad.concat(batch_score_datas)
    
    sc.pp.neighbors(score_data, random_state=42)
    sc.tl.umap(score_data, random_state=42)
    sc.tl.pca(score_data, random_state=42, n_comps=2)
    
    return score_data

# Visualize the scores

In [ ]:
# scores for all regions
selected_score = 'aR'
embedded = umap_dataframe(normed_pseudobulk, selected_score)
embedded.obs['pc1'] = embedded.obsm['X_pca'][:,0]
embedded.obs['pc2'] = embedded.obsm['X_pca'][:,1]

tc.utils.merge_annotation(embedded,'source-State',mapping={
    'normal':['Becker: Normal','SlideSeq: normal','scRNA: normal','Chen: NL','Zheng: normal','Khaliq: Normal','Pelka: normal','Joanito_3p: Normal','Joanito_5p: Normal',],
    'malignant':['Pelka: MMRd','Pelka: MMRp','Becker: Adenocarcinoma','Che: CRC', 'Chen: MSI-H', 'Chen: MSS', 'Khaliq: Tumor', 'Zheng: carcinoma', 'scRNA: malignant (3weeks)', 'scRNA: malignant (9weeks)','Joanito_3p: MSS','Joanito_5p: MSS','Joanito_3p: MSI-H','Joanito_5p: MSI-H',],
    'unaffected':['Becker: Unaffected',],
    'premalignant':['SlideSeq: premalignant','scRNA: premalignant','Chen: AD','Chen: SER','Zheng: adenoma','Becker: Polyp'],
    'other':['Che: LM', 'Chen: UNC', 'Zheng: para-cancer'],
},result_key='simple-State')
embedded.obs['simple-State'] = embedded.obs['simple-State'].astype(pd.CategoricalDtype(['normal','unaffected','premalignant','malignant','other'], ordered=True))
embedded.obs['source'] = embedded.obs['source'].astype(pd.CategoricalDtype(['Pelka','Becker','Che','Chen','Zheng','Joanito_3p','Joanito_5p','Khaliq','scRNA','SlideSeq'], ordered=True))

tc.utils.merge_annotation(embedded,'source-State',mapping={
    'human normal':['Becker: Normal','Chen: NL','Zheng: normal','Khaliq: Normal','Pelka: normal','Joanito_3p: Normal','Joanito_5p: Normal',],
    'mouse normal':['SlideSeq: normal','scRNA: normal',],
    'human tumor':['Pelka: MMRd','Pelka: MMRp','Becker: Adenocarcinoma','Che: CRC', 'Chen: MSI-H', 'Chen: MSS','Khaliq: Tumor', 'Zheng: carcinoma', 'Joanito_3p: MSS','Joanito_5p: MSS','Joanito_3p: MSI-H','Joanito_5p: MSI-H',],
    'mouse malignant':['scRNA: malignant (3weeks)', 'scRNA: malignant (9weeks)'],
    'human unaffected':['Becker: Unaffected',],
    'human polyp':['Chen: AD','Chen: SER','Zheng: adenoma','Becker: Polyp',],
    'mouse premalignant':['SlideSeq: premalignant','scRNA: premalignant',],
    'human other':['Che: LM', 'Chen: UNC', 'Zheng: para-cancer'],
},result_key='Species-Label')
embedded.obs['Species-Label'] = embedded.obs['Species-Label'].astype(pd.CategoricalDtype(['mouse normal', 'mouse premalignant', 'mouse malignant', 'human normal', 'human unaffected', 'human polyp', 'human tumor', 'human other',], ordered=True))

tc.utils.merge_annotation(embedded,'source-State',mapping={
    'human normal':['Becker: Normal','Chen: NL','Zheng: normal','Pelka: normal','Joanito_3p: Normal','Joanito_5p: Normal','Khaliq: Normal',],
    'mouse normal':['SlideSeq: normal','scRNA: normal',],
    'human tumor MMRd':['Pelka: MMRd','Chen: MSI-H','Joanito_3p: MSI-H','Joanito_5p: MSI-H',],
    'human tumor MMRp':['Pelka: MMRp','Chen: MSS','Joanito_3p: MSS','Joanito_5p: MSS',],
    'human tumor general':['Becker: Adenocarcinoma','Che: CRC', 'Khaliq: Tumor', 'Zheng: carcinoma', ],
    'mouse malignant':['scRNA: malignant (3weeks)', 'scRNA: malignant (9weeks)'],
    'human unaffected':['Becker: Unaffected',],
    'human polyp general':['Zheng: adenoma','Becker: Polyp',],
    'mouse premalignant':['SlideSeq: premalignant','scRNA: premalignant',],
    'human other':['Che: LM', 'Chen: UNC', 'Zheng: para-cancer'],
},result_key='Tumor-Subtype')
embedded.obs['Tumor-Subtype'] = embedded.obs['Tumor-Subtype'].astype(pd.CategoricalDtype(['mouse normal', 'mouse premalignant', 'mouse malignant', 'human normal', 'human unaffected', 'Chen: AD','Chen: SER', 'human polyp general', 'human tumor MMRd', 'human tumor MMRp', 'human tumor general', 'human other',], ordered=True))

embedded.obs['ordered-State'] = embedded.obs['source-State'].astype(pd.CategoricalDtype([
    'Zheng: normal','Chen: NL','Becker: Normal','Pelka: normal','Joanito_3p: Normal','Joanito_5p: Normal','Khaliq: Normal', 'SlideSeq: normal','scRNA: normal',# normal
    'Becker: Unaffected', # unaffected
    'Chen: SER','Chen: AD','Becker: Polyp','Zheng: adenoma','SlideSeq: premalignant','scRNA: premalignant', # premalignant
    'Zheng: carcinoma',  'Becker: Adenocarcinoma','Che: CRC', 'Pelka: MMRd','Pelka: MMRp','Chen: MSI-H', 'Chen: MSS','Joanito_3p: MSI-H','Joanito_3p: MSS','Joanito_5p: MSI-H','Joanito_5p: MSS','Khaliq: Tumor', 'scRNA: malignant (3weeks)', 'scRNA: malignant (9weeks)', # malignant
    'Chen: UNC', 'Zheng: para-cancer', 'Che: LM', # other
], ordered=True))

In [ ]:
loading_df = pd.Series(embedded.varm['PCs'][:,0],index=embedded.var.index).reset_index().rename(columns={'index':'region_score',0:'pc1'})
loading_df['region_score'] = loading_df['region_score'].str.split('_').str[0]

enriched_color = (1.0, 0.07058823529411765, 0.09019607843137255)
depleted_color = (0.30196078431372547, 0.5215686274509804, 0.7098039215686275)
null_color = (0.9,0.9,0.9)

loading_df = loading_df.sort_values('pc1',ascending=False)

pc1_min, pc1_max = loading_df['pc1'].min(), loading_df['pc1'].max()
pc1_absmax = max(abs(pc1_min),abs(pc1_max))
norm = Normalize(vmin=-pc1_absmax, vmax=pc1_absmax)
cmap = LinearSegmentedColormap.from_list('pc1', [(0,depleted_color),(0.5,null_color),(1,enriched_color)])
mapper = ScalarMappable(norm=norm, cmap=cmap)
colors = mapper.to_rgba(loading_df['pc1'])

fig,axs = tc.pl.subplots(axsize=[3,3])
y_pos = np.arange(len(loading_df))
axs[0,0].barh(y_pos, loading_df['pc1'], align='center', color=colors)
axs[0,0].set_yticks(y_pos, labels=loading_df['region_score'])
axs[0,0].invert_yaxis()  # labels read top-to-bottom
axs[0,0].set_xlabel('pc1 loadings');

fig.savefig(f'{figures_folder}/epi_{selected_score}_region_score_pca_pc1_loadings.pdf',bbox_inches='tight')

In [ ]:
region_loading_order = pd.Index(loading_df['region_score'])

In [ ]:
all_groups = embedded.obs['Species-Label'].cat.categories
all_groups_by_species = { species: all_groups[all_groups.str.startswith(species)] for species in ['human','mouse'] }
enr_list = []
for species, species_groups in all_groups_by_species.items():
    for region_score in embedded.var_names:
        enr_list.append(tc.tl.enrichments(embedded, region_score, "Species-Label", method='welch', value_location='X', reference_group=f'{species} normal', restrict_groups=species_groups).rename(columns={region_score:'region_score'}))
        premalignant_group, malignant_group = ['mouse premalignant', 'mouse malignant'] if species == 'mouse' else ['human polyp', 'human tumor']
        enr_list.append(tc.tl.enrichments(embedded, region_score, "Species-Label", method='welch', value_location='X', reference_group=premalignant_group, restrict_groups=[premalignant_group, malignant_group]).rename(columns={region_score:'region_score'}))
enr = pd.concat(enr_list)
enr.loc[enr['Species-Label'] == 'mouse normal VS rest', 'Species-Label'] = 'mouse normal VS mouse rest'
enr.loc[enr['Species-Label'] == 'human normal VS rest', 'Species-Label'] = 'human normal VS human rest'
comparison_order = ['mouse normal VS mouse rest', 'human normal VS human rest', 'mouse premalignant VS mouse normal', 'human polyp VS human normal', 'mouse malignant VS mouse normal', 'human tumor VS human normal', 'mouse malignant VS mouse premalignant', 'human tumor VS human polyp', 'human other VS human normal', 'human unaffected VS human normal']
enr = enr[enr['Species-Label'].isin(comparison_order)].copy()
enr['Species-Label'] = enr['Species-Label'].astype(pd.CategoricalDtype(comparison_order, ordered=True))
enr['region_score'] = enr['region_score'].astype(pd.CategoricalDtype([f'{r}_{selected_score}score' for r in region_loading_order], ordered=True))
enr['p_welch_fdr_bh'] = multipletests(enr['p_welch'], alpha=0.05, method='fdr_bh')[1]
fig = tc.pl.significances(enr, 'p_welch_fdr_bh', 'region_score', 'Species-Label', annotate_pvalues=False);
fig.savefig(f'{figures_folder}/epi_{selected_score}_region_score_significance_welch_reordered.pdf',bbox_inches='tight')

In [ ]:
all_groups = embedded.obs['Species-Label'].cat.categories
all_groups_by_species = { species: all_groups[all_groups.str.startswith(species)] for species in ['human','mouse'] }
enr_list = []
for species, species_groups in all_groups_by_species.items():
    for iA,gA in enumerate(species_groups):
        for iB,gB in enumerate(species_groups):
            if iA != iB:
                enr_AB = tc.tl.enrichments(embedded, 'pc1', "Species-Label", reference_group=gA, restrict_groups=[gA,gB])
                enr_list.append(enr_AB[~enr_AB['Species-Label'].str.endswith(' VS rest')])
enr = pd.concat(enr_list)
enr['p_mwu_fdr_bh'] = multipletests(enr['p_mwu'], alpha=0.05, method='fdr_bh')[1]
for k,v in enr['Species-Label'].str.split(' VS ', expand=True).rename(columns={0: 'gA', 1: 'gB'}).items():
    enr[k] = v.astype(pd.CategoricalDtype(['mouse normal', 'mouse premalignant', 'mouse malignant', 'human normal', 'human unaffected', 'human polyp', 'human tumor', 'human other'], ordered=True))

In [ ]:
x_dir = 'Species-Label'

fig,axs=tc.pl.subplots(2,1,axsize=(8,5),x_padding=2,y_padding=2)

sns.scatterplot(data=embedded.obs, x='pc1', y='pc2', ax=axs[0,0], hue='simple-State', style='species',)
axs[0,0].legend(bbox_to_anchor=(1, 1), loc='upper left')

ax = axs[0,1]
sns.boxplot(data=embedded.obs, x=x_dir, y="pc1", ax=ax, hue='simple-State', dodge=False, showfliers=False)
sns.stripplot(data=embedded.obs, x=x_dir, y="pc1", ax=ax, color=".25")
sel = enr.groupby(enr[x_dir].str.split(' VS ').map(np.sort).map(lambda x: f'{x[0]} VS {x[1]}'))['p_mwu_fdr_bh'].min()
pairs = sel.index.str.split(' VS ')
pvals = sel.to_numpy()
annotator = Annotator(ax, pairs=pairs, data=embedded.obs, x=x_dir, y="pc1", verbose=False)
annotator.annotate_custom_annotations(np.where(pvals > 0.05, 'ns', [f'FDR={np.format_float_scientific(p, unique=False, exp_digits=1, precision=1)}' for p in pvals]))
cats = embedded.obs[x_dir].cat.categories
ax.set_xticks(np.arange(len(cats)))
ax.set_xticklabels(cats, rotation=90,va='top',ha='center');
ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
fig.savefig(f'{figures_folder}/epi_{selected_score}_region_score_pca.pdf',bbox_inches='tight')

In [ ]:
x_dir = 'ordered-State'
for i_what,what in enumerate(['simple-State','source']):
    fig,axs=tc.pl.subplots(1,1,axsize=(8,5),x_padding=2,y_padding=2)
    ax = axs[0,0]
    sns.boxplot(data=embedded.obs, x=x_dir, y="pc1", ax=ax, hue=what, dodge=False, showfliers=False)
    sns.stripplot(data=embedded.obs, x=x_dir, y="pc1", ax=ax, color=".25")
    cats = embedded.obs[x_dir].cat.categories
    ax.set_xticks(np.arange(len(cats)))
    ax.set_xticklabels(cats, rotation=90,va='top',ha='center');
    ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
    fig.savefig(f'{figures_folder}/epi_{selected_score}_region_score_pca_{what}.pdf',bbox_inches='tight')

# relationship between region scores and CMS classifications

In [ ]:
embedded.obs['CMS_classes'] = embedded.obs.index.map(CMS_classes['predictedCMS']).astype(pd.CategoricalDtype(sorted([c for c in CMS_classes['predictedCMS'].unique() if str(c).startswith('CMS')]),ordered=True))

In [ ]:
sub = embedded
sub = sub[sub.obs['species'] != 'mouse']
sub = sub[~sub.obs['simple-State'].isin(['normal','unaffected'])]

In [ ]:
enr = tc.tl.enrichments(sub[~sub.obs["CMS_classes"].isna()], None, "CMS_classes", value_location='X', method='welch', )
enr['region_score'] = enr['value'].astype(pd.CategoricalDtype([f'{r}_{selected_score}score' for r in region_loading_order], ordered=True))
fig = tc.pl.significances(enr, 'p_welch_fdr_bh', 'region_score', 'CMS_classes', );
fig.savefig(f'{figures_folder}/epi_{selected_score}_region_score_CMS_enrichment_welch.pdf',bbox_inches='tight')